In [18]:
from google.colab import files
uploaded = files.upload()

Saving dataset-of-00s.csv to dataset-of-00s (1).csv
Saving dataset-of-10s.csv to dataset-of-10s (1).csv
Saving dataset-of-60s.csv to dataset-of-60s (1).csv
Saving dataset-of-70s.csv to dataset-of-70s (1).csv
Saving dataset-of-80s.csv to dataset-of-80s (1).csv
Saving dataset-of-90s.csv to dataset-of-90s (1).csv


In [19]:
import os
import glob
import pandas as pd
import numpy as np 

np.random.seed(20)

In [ ]:
# Define working directory
# os.chdir(r"C:\Users\Cristina Bardan\Desktop\Repositories\FinalProject-Spotify\Original_data")

In [20]:
# Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['dataset-of-00s.csv',
 'dataset-of-60s (1).csv',
 'dataset-of-60s.csv',
 'dataset-of-70s.csv',
 'dataset-of-80s (1).csv',
 'dataset-of-10s (1).csv',
 'dataset-of-90s (1).csv',
 'dataset-of-80s.csv',
 'dataset-of-90s.csv',
 'dataset-of-00s (1).csv',
 'dataset-of-70s (1).csv',
 'dataset-of-10s.csv']

In [21]:
# Combine all files in the list and export as CSV
data = pd.concat([pd.read_csv(f) for f in all_filenames ])
data.reset_index(drop=True, inplace=True)
data

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Lucky Man,Montgomery Gentry,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,0.578,0.471,4,-7.270,1,0.0289,0.368000,0.000000,0.159,0.5320,133.061,196707,4,30.88059,13,1
1,On The Hotline,Pretty Ricky,spotify:track:1zyqZONW985Cs4osz9wlsu,0.704,0.854,10,-5.477,0,0.1830,0.018500,0.000000,0.148,0.6880,92.988,242587,4,41.51106,10,1
2,Clouds Of Dementia,Candlemass,spotify:track:6cHZf7RbxXCKwEkgAZT4mY,0.162,0.836,9,-3.009,1,0.0473,0.000111,0.004570,0.174,0.3000,86.964,338893,4,65.32887,13,0
3,"Heavy Metal, Raise Hell!",Zwartketterij,spotify:track:2IjBPp2vMeX7LggzRN3iSX,0.188,0.994,4,-3.745,1,0.1660,0.000007,0.078400,0.192,0.3330,148.440,255667,4,58.59528,9,0
4,I Got A Feelin',Billy Currington,spotify:track:1tF370eYXUcWwkIvaq3IGz,0.630,0.764,2,-4.353,1,0.0275,0.363000,0.000000,0.125,0.6310,112.098,193760,4,22.62384,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82207,Lotus Flowers,Yolta,spotify:track:4t1TljQWJ6ZuoSY67zVvBI,0.172,0.358,9,-14.430,1,0.0342,0.886000,0.966000,0.314,0.0361,72.272,150857,4,24.30824,7,0
82208,Calling My Spirit,Kodak Black,spotify:track:2MShy1GSSgbmGUxADNIao5,0.910,0.366,1,-9.954,1,0.0941,0.099600,0.000000,0.261,0.7400,119.985,152000,4,32.53856,8,1
82209,Teenage Dream,Katy Perry,spotify:track:55qBw1900pZKfXJ6Q9A2Lc,0.719,0.804,10,-4.581,1,0.0355,0.013200,0.000003,0.139,0.6050,119.999,227760,4,20.73371,7,1
82210,Stormy Weather,Oscar Peterson,spotify:track:4o9npmYHrOF1rUxxTVH8h4,0.600,0.177,7,-16.070,1,0.0561,0.989000,0.868000,0.149,0.5600,120.030,213387,4,21.65301,14,0


In [22]:
data2 = data.copy()
data2.loudness = (data2.loudness +50)/max(data2.loudness +50)
data2.duration_ms = (data2.duration_ms / 1000)/max(data2.duration_ms / 1000)
data2.key = (data2.key)/max(data2.key)
data2.tempo = (data2.tempo)/max(data2.tempo)
data2.time_signature = (data2.time_signature)/max(data2.time_signature)
data2.chorus_hit = (data2.chorus_hit)/max(data2.chorus_hit)
data2.sections = (data2.sections)/max(data2.sections)

data2

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Lucky Man,Montgomery Gentry,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,0.578,0.471,0.363636,0.795065,1,0.0289,0.368000,0.000000,0.159,0.5320,0.551153,0.047169,0.8,0.071288,0.076923,1
1,On The Hotline,Pretty Ricky,spotify:track:1zyqZONW985Cs4osz9wlsu,0.704,0.854,0.909091,0.828427,0,0.1830,0.018500,0.000000,0.148,0.6880,0.385166,0.058171,0.8,0.095828,0.059172,1
2,Clouds Of Dementia,Candlemass,spotify:track:6cHZf7RbxXCKwEkgAZT4mY,0.162,0.836,0.818182,0.874349,1,0.0473,0.000111,0.004570,0.174,0.3000,0.360214,0.081265,0.8,0.150812,0.076923,0
3,"Heavy Metal, Raise Hell!",Zwartketterij,spotify:track:2IjBPp2vMeX7LggzRN3iSX,0.188,0.994,0.363636,0.860654,1,0.1660,0.000007,0.078400,0.192,0.3330,0.614854,0.061308,0.8,0.135267,0.053254,0
4,I Got A Feelin',Billy Currington,spotify:track:1tF370eYXUcWwkIvaq3IGz,0.630,0.764,0.181818,0.849341,1,0.0275,0.363000,0.000000,0.125,0.6310,0.464322,0.046463,0.8,0.052227,0.059172,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82207,Lotus Flowers,Yolta,spotify:track:4t1TljQWJ6ZuoSY67zVvBI,0.172,0.358,0.818182,0.661841,1,0.0342,0.886000,0.966000,0.314,0.0361,0.299358,0.036175,0.8,0.056116,0.041420,0
82208,Calling My Spirit,Kodak Black,spotify:track:2MShy1GSSgbmGUxADNIao5,0.910,0.366,0.090909,0.745125,1,0.0941,0.099600,0.000000,0.261,0.7400,0.496991,0.036449,0.8,0.075115,0.047337,1
82209,Teenage Dream,Katy Perry,spotify:track:55qBw1900pZKfXJ6Q9A2Lc,0.719,0.804,0.909091,0.845099,1,0.0355,0.013200,0.000003,0.139,0.6050,0.497049,0.054616,0.8,0.047864,0.041420,1
82210,Stormy Weather,Oscar Peterson,spotify:track:4o9npmYHrOF1rUxxTVH8h4,0.600,0.177,0.636364,0.631326,1,0.0561,0.989000,0.868000,0.149,0.5600,0.497177,0.051169,0.8,0.049986,0.082840,0


In [ ]:
data2.columns

Index(['track', 'artist', 'uri', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'chorus_hit',
       'sections', 'target'],
      dtype='object')

In [23]:
y = data2.target
x = data2[['danceability', 'energy', 'key', 'loudness','mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo', 'duration_ms', 'time_signature', 'chorus_hit','sections']]

In [ ]:
print(x.shape, y.shape)

(41106, 15) (41106,)


In [24]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=12)

### With RANDOM FOREST

In [35]:
# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, max_depth=25) 
model = model.fit(x_train, y_train)

In [36]:
print(f"Training Data Score: {model.score(x_train, y_train)}")
print(f"Testing Data Score: {model.score(x_test, y_test)}")

Training Data Score: 0.9957346048427642
Testing Data Score: 0.9388410451029047


In [37]:
# Validation of feature importance
importances = model.feature_importances_
importances

array([0.10229701, 0.085402  , 0.02819235, 0.07420612, 0.01065278,
       0.07324727, 0.11112636, 0.1756033 , 0.0496211 , 0.0681226 ,
       0.05244977, 0.07545421, 0.00729069, 0.048534  , 0.03780044])

In [38]:
# We can sort the features by their importance

sorted(zip(model.feature_importances_, x.columns), reverse=True)

[(0.17560330323965012, 'instrumentalness'),
 (0.11112636216648088, 'acousticness'),
 (0.10229701168487333, 'danceability'),
 (0.08540199702761168, 'energy'),
 (0.0754542056029328, 'duration_ms'),
 (0.07420612400276559, 'loudness'),
 (0.07324727353857342, 'speechiness'),
 (0.06812259814995575, 'valence'),
 (0.05244976754212371, 'tempo'),
 (0.04962110436687015, 'liveness'),
 (0.0485339952313036, 'chorus_hit'),
 (0.0378004354563939, 'sections'),
 (0.028192348591277342, 'key'),
 (0.010652782855537335, 'mode'),
 (0.007290690543650239, 'time_signature')]

In [29]:
# Save the model
from google.colab import drive
drive.mount('/content/gdrive')
path = F"/content/gdrive/My Drive/rf.h5" 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
import pickle
with open(path, 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# import pickle
# with open('../Models/h5/rf.h5', 'wb') as f:
#     pickle.dump(model, f)

In [ ]:
# with open('../Models/h5/rf.h5','rb') as fp:
#     thing = pickle.load(fp)
# thing